In [1]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
from tqdm import tqdm

### Setting a device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Initializing the Hyperparameters

In [13]:
num_classes = 10
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5

### Load Pretrain Model

In [4]:
class Identity(nn.Module):
  def __init__(self):
    super(Identity, self).__init__()

  def forward(self, x):
    return x

In [14]:
model = torchvision.models.vgg16(pretrained=True)

# Freezing the parameters with no_grad
for param in model.parameters():
  param.requires_grad = False

In [15]:
model.avgpool = Identity() # avgpool[1] = Identity()

# Now we are changing the FC layer ... so for this, requires_grad=True
model.classifier = nn.Sequential(
    nn.Linear(512, 100), nn.ReLU(), nn.Dropout(p=0.5), nn.Linear(100, num_classes)
) # since the no of kernels/filters in vgg is 512

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Load Data

In [11]:
train_dataset = datasets.CIFAR10(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


### Loss & Optimizer

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Training the Model

In [17]:
for epoch in tqdm(range(num_epochs)):
  losses = []

  for ind, (data, target) in enumerate(train_loader):
    data = data.to(device=device)
    target = target.to(device=device)
    
    # forward
    scores = model(data)
    loss = criterion(scores, target)
    losses.append(loss)

    # backward
    optimizer.zero_grad() # set all the gradients to 0 for each batch
    loss.backward()

    # gradient descent or adam step - updates the parameters
    optimizer.step()

  mean_loss = sum(losses)/len(losses)

  print(f"\nLoss at Epoch {epoch+1} is {mean_loss}")

 20%|██        | 1/5 [00:22<01:28, 22.09s/it]


Loss at Epoch 1 is 1.6972723007202148


 40%|████      | 2/5 [00:44<01:06, 22.09s/it]


Loss at Epoch 2 is 1.358735203742981


 60%|██████    | 3/5 [01:06<00:44, 22.10s/it]


Loss at Epoch 3 is 1.2767080068588257


 80%|████████  | 4/5 [01:28<00:22, 22.11s/it]


Loss at Epoch 4 is 1.2333495616912842


100%|██████████| 5/5 [01:50<00:00, 22.11s/it]


Loss at Epoch 5 is 1.203576683998108


### Testing the Model

In [18]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval() # Let the model know that this is evaluation mode

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      
      # Which class has the max value
      _, pred = scores.max(dim=1)
      num_correct += (pred == y).sum()
      num_samples += pred.size(0) # 64

    print(f"Got {num_correct} / {num_samples} with accuracy {(float(num_correct)/float(num_samples)*100):.2f}%")

In [19]:
check_accuracy(test_loader, model)

Got 6021 / 10000 with accuracy 60.21%
